In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
import requests
from bs4 import BeautifulSoup
import matplotlib as mlp
mlp.use("TKAgg")
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
mlp.rcParams.update({'font.family': "Open Sans", 'font.size' : 16})

In [2]:
#import social security names database from Kaggle in int form
names = pd.read_csv("/Users/kejinglin/Desktop/personal/NationalNames.csv", dtype = {'Count': np.int32})
names = names.fillna(0)
names.head()

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746


In [3]:
namechart = names.groupby(['Name', 'Gender'], as_index = False)['Count'].sum()
namechart.head(5)

,Name,Gender,Count
0,Aaban,M,72
1,Aabha,F,21
2,Aabid,M,5
3,Aabriella,F,10
4,Aadam,M,196


In [4]:
namechartdiff = namechart.reset_index().pivot('Name', 'Gender', 'Count')
namechartdiff = namechartdiff.fillna(0)
namechartdiff["Mpercent"] = ((namechartdiff["M"] - namechartdiff["F"])/(namechartdiff["M"] + namechartdiff["F"]))
namechartdiff['gender'] = np.where(namechartdiff['Mpercent'] > 0.001, 'male', 'female')
namechartdiff.head()

Gender,F,M,Mpercent,gender
Name,,,,
Aaban,0.0,72.0,1.0,male
Aabha,21.0,0.0,-1.0,female
Aabid,0.0,5.0,1.0,male
Aabriella,10.0,0.0,-1.0,female
Aadam,0.0,196.0,1.0,male


In [5]:
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2))
X = char_vectorizer.fit_transform(namechartdiff.index)
X = X.tocsc()
y = (namechartdiff.gender == 'male').values.astype(np.int)
print(X)

  (0, 0)	1
  (1, 0)	1
  (2, 0)	1
  (3, 0)	1
  (4, 0)	1
  (5, 0)	1
  (6, 0)	1
  (7, 0)	1
  (8, 0)	1
  (9, 0)	1
  (10, 0)	1
  (11, 0)	1
  (12, 0)	1
  (13, 0)	1
  (14, 0)	1
  (15, 0)	1
  (16, 0)	1
  (17, 0)	1
  (18, 0)	1
  (19, 0)	1
  (20, 0)	1
  (21, 0)	1
  (22, 0)	1
  (23, 0)	1
  (24, 0)	1
  :	:
  (64876, 616)	1
  (67519, 616)	1
  (67520, 616)	1
  (67521, 616)	1
  (72287, 616)	1
  (73357, 616)	1
  (73358, 616)	1
  (76118, 616)	1
  (81252, 616)	1
  (81253, 616)	1
  (81254, 616)	1
  (81255, 616)	1
  (81256, 616)	1
  (83577, 616)	1
  (88001, 616)	1
  (88002, 616)	1
  (88270, 616)	1
  (91145, 616)	1
  (91333, 616)	1
  (91334, 616)	1
  (91335, 616)	1
  (91336, 616)	1
  (91604, 616)	1
  (92000, 616)	1
  (93888, 616)	1


In [6]:
itrain, itest = train_test_split(range(namechartdiff.shape[0]), train_size=0.7)
mask=np.ones(namechartdiff.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

In [7]:
Xtrainthis=X[mask]
Ytrainthis=y[mask]
Xtestthis=X[~mask]
Ytestthis=y[~mask]
clf = MultinomialNB(alpha = 1)
clf.fit(Xtrainthis, Ytrainthis)
training_accuracy = clf.score(Xtrainthis,Ytrainthis)
test_accuracy = clf.score(Xtestthis,Ytestthis)
        
print(training_accuracy)
print(test_accuracy)

0.7421411399531359
0.736358149607697


In [8]:
def lookup(x):
    str(x)
    new = char_vectorizer.transform([x])
    y_pred = clf.predict(new)
    if (y_pred == 1):
        return 'M'
    else:
        return 'F'

In [10]:
import pandas as pd
import numpy as np
import re

tweet_unique = pd.read_csv(r'/Users/kejinglin/Desktop/personal/mydata/tweet_clean_sem.csv',encoding='latin-1')
tweet_unique.describe()

,m_content_url,m_content_id,r_comment_num,r_like_num,r_trans_num,verified,u_id,u_fans,u_follow,sentiment_keras,sentiment_lstm
count,0.0,3.358000e+03,3358.000000,3358.000000,3358.000000,3358.000000,3.358000e+03,3.358000e+03,3358.000000,3.358000e+03,3358.000000
mean,NaN,1.269764e+18,876.355867,9040.721858,2626.723943,0.765337,1.148524e+17,1.510382e+06,7170.366885,5.164667e-01,2.190292
std,NaN,3.673728e+16,2762.278035,25413.313668,6044.815140,0.423851,3.167638e+17,4.568415e+06,27375.859921,4.167977e-01,1.550212
min,NaN,1.217043e+18,101.000000,135.000000,103.000000,0.000000,1.200000e+01,3.400000e+01,0.000000,1.676390e-11,0.000000
25%,NaN,1.242524e+18,157.000000,1461.000000,500.000000,1.000000,3.423321e+07,1.215850e+05,536.000000,5.021494e-02,1.000000
50%,NaN,1.254465e+18,271.000000,3030.500000,1119.000000,1.000000,1.979696e+08,3.692000e+05,1304.000000,5.631192e-01,2.000000
75%,NaN,1.294606e+18,570.750000,7152.000000,2354.000000,1.000000,1.383060e+09,1.184751e+06,2717.000000,9.543029e-01,4.000000
max,NaN,1.360644e+18,43038.000000,396364.000000,101008.000000,1.000000,1.211790e+18,1.293520e+08,594833.000000,9.999977e-01,4.000000


In [11]:
from pandas.core.frame import DataFrame
import os
result=[]
for i in tweet_unique['u_nickname']:
    result.append(lookup(i))

name_gender=DataFrame(result)
name_gender.to_excel('/Users/kejinglin/Desktop/personal/mydata/name_gender_notunique.xls')
#name_gender.to_csv('/Users/kejinglin/Desktop/personal/mydata/name_gender_notunique.csv')